In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
import itertools
import pickle
import logging

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")
print(df.shape)

columns_to_drop = ['VERBALE', 'DATA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro',
                    'Totale', 'Tot Volta cranica', 'Tot Base cranica', 
                    'Tot Neuroc.', 'Tot Splancnoc.', 'Tot Testa',
                    'Tot Tratto toracico', 'Tot Tratto lombare', 'Tot Rachide',
                    ' Totale coste', 'Sterno in toto', 'Tot Bacino', 'I costa dx', 'II costa dx',
                    'III costa dx', 'IV costa dx', 'V costa dx', 'VI costa dx', 'VII costa dx', 
                    'VIII costa dx', 'IX costa dx', 'X costa dx', 'XI costa dx', 'XII costa dx',
                    'I costa sx', 'II costa sx', 'III costa sx', 'IV costa sx', 'V costa sx', 
                    'VI costa sx', 'VII costa sx', 'VIII costa sx', 'IX costa sx', 
                    'X costa sx', 'XI costa sx', 'XII costa sx']

X = df.drop(columns=columns_to_drop)
print(X.shape)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
print(X.shape)

(130, 367)
(130, 326)
(130, 274)


In [3]:
random_seed = 3355825

In [4]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [12]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('../logs/lof_nested_cv.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [22]:
def nested_cv_extended(X, random_seed, decomposition, n_outer_folds=5, n_inner_folds=3, n_components=5, mod_selection_score=accuracy_score, positive_class=0):
    n_neighborss = [35, 40, 45, 50]
    contaminations = np.linspace(0.01, 0.5, 15) # 50 for PCA and TruncatedSVD, 15 for FastICA
    metrics = ['euclidean', 'minkowski', 'manhattan', 'cosine']
    scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': '', 'components': 0, 'scaler': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    best_overall_accuracy = 0
    best_overall_params = {'n_neighbors': 0, 'contamination': 0, 'metric': '', 'components': 0, 'scaler': ''}

    y = X['Mezzo'].values
    y = np.array([0. if x == positive_class else 1. for x in y], dtype=float)
    
    X = X.drop(columns='Mezzo').values

    outer_cv = StratifiedKFold(n_splits=n_outer_folds, shuffle=True, random_state=random_seed)
        
    for outer_cv_number, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        inner_cv = StratifiedKFold(n_splits=n_inner_folds, shuffle=True, random_state=random_seed)

        best_score = 0
        best_decomposition = decomposition
        best_scaler = StandardScaler()
        for inner_cv_number, (trainval_idx, valid_idx) in enumerate(inner_cv.split(X_train, y_train)):
            for params in itertools.product(n_neighborss, contaminations, metrics, scalers):
                scaler = params[3]
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                X_trainval_scaled = scaler.fit_transform(X_trainval)
                X_valid_scaled = scaler.transform(X_valid)
                
                decomp = decomposition(n_components=n_components)
                    
                X_trainval_reduced = decomp.fit_transform(X_trainval_scaled)
                X_valid_reduced = decomp.transform(X_valid_scaled)

                idxs_neg = np.where(y_trainval == 1)[0]
                X_trainval_reduced = np.delete(X_trainval_reduced, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)
                    
                clf.fit(X_trainval_reduced)
                    
                pred_values = clf.predict(X_valid_reduced)
                true_values = [1 if y == 0 else -1 for y in y_valid]
                    
                score = mod_selection_score(true_values, pred_values)
                curr_params = {
                        'n_neighbors': params[0],
                        'contamination': params[1],
                        'metric': params[2],
                        'components': n_components,
                        'scaler': params[3]
                }

                logging.info(f"inner cv number: {inner_cv_number}, {mod_selection_score.__name__}: {score}, with params: {curr_params}")
                        
                if score > best_score:
                    best_score = score
                    best_decomposition = decomp
                    best_scaler = scaler
                    best_params = curr_params

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)
        y_train = np.delete(y_train, idxs_neg)

        X_train_scaled = best_scaler.fit_transform(X_train)
        X_test_scaled = best_scaler.transform(X_test)

        X_train_reduced = best_decomposition.transform(X_train_scaled)
        X_test_reduced = best_decomposition.transform(X_test_scaled)

        clf = LocalOutlierFactor(n_neighbors=best_params['n_neighbors'], contamination=best_params['contamination'], metric=best_params['metric'], novelty=True)
        clf.fit(X_train_reduced)

        pred_values = clf.predict(X_test_reduced)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy = accuracy_score(true_values, pred_values)
        precision = precision_score(true_values, pred_values, zero_division=0.0)
        recall = recall_score(true_values, pred_values)
        f1 = f1_score(true_values, pred_values)

        if accuracy > best_overall_accuracy:
            best_overall_accuracy = accuracy
            best_overall_params = best_params

        logging.info(f"outer cv number: {outer_cv_number}, accuracy: {accuracy}, precision: {precision}, recall: {recall}, f1: {f1} with params: {best_params}")

        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

    return {
        'algorythm': 'LocalOutlierFactor',
        'best n_neighbors': best_overall_params['n_neighbors'],
        'best contamination': best_overall_params['contamination'],
        'best metric': best_overall_params['metric'],
        'decomposition': decomposition,
        'n_components': best_overall_params['components'],
        'best scaler': best_overall_params['scaler'],
        'score used for model selection': mod_selection_score.__name__,
        'method used for model selection': 'nested cv',
        'accuracy mean': np.mean(accuracy_scores) * 100,
        'accuracy std': np.std(accuracy_scores) * 100,
        'precision mean': np.mean(precision_scores) * 100,
        'precision std': np.std(precision_scores) * 100,
        'recall mean': np.mean(recall_scores) * 100,
        'recall std': np.std(recall_scores) * 100,
        'f1 mean': np.mean(f1_scores) * 100,
        'f1 std': np.std(f1_scores) * 100,
        'best overall accuracy': best_overall_accuracy * 100,
        'class': positive_class
    }

In [18]:
results = nested_cv_extended(X, random_seed, n_outer_folds=13, n_inner_folds=6, pca_components=100, mod_selection_score=accuracy_score, positive_class=0)
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_neighbors,best contamination,best metric,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.0,0


In [19]:
for pca_comps in np.arange(95, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, n_outer_folds=13, n_inner_folds=6, pca_components=pca_comps, mod_selection_score=accuracy_score, positive_class=0))

In [22]:
scores_df

,algorythm,best n_neighbors,best contamination,best metric,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.0,0
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.0,0
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.0,0
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.0,0
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.0,0
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.0,0
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.0,0
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.0,0
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.0,0


In [27]:
for pca_comps in np.arange(100, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, n_outer_folds=13, n_inner_folds=6, pca_components=pca_comps, mod_selection_score=f1_score, positive_class=0))

scores_df

,algorythm,best n_neighbors,best contamination,best metric,best pca components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.0,0
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.0,0
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.0,0
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.0,0
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.0,0
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.0,0
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.0,0
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.0,0
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.0,0
9,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),f1_score,nested cv,69.230769,9.166443,71.391941,6.264593,73.333333,17.492367,71.258741,9.420480,90.0,0


In [32]:
scores_df['decomposition'] = PCA
scores_df = scores_df.rename(columns={'best pca components': 'n_components'})
scores_df

,algorythm,best n_neighbors,best contamination,best metric,n_components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class,decomposition
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.0,0,<class 'sklearn.decomposition._pca.PCA'>
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.0,0,<class 'sklearn.decomposition._pca.PCA'>
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
9,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),f1_score,nested cv,69.230769,9.166443,71.391941,6.264593,73.333333,17.492367,71.258741,9.420480,90.0,0,<class 'sklearn.decomposition._pca.PCA'>


In [33]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('../logs/lof_svd_nested_cv.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [36]:
for n_comps in np.arange(100, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, TruncatedSVD, n_outer_folds=13, n_inner_folds=6, n_components=n_comps))

scores_df

,algorythm,best n_neighbors,best contamination,best metric,n_components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class,decomposition
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.0,0,<class 'sklearn.decomposition._pca.PCA'>
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.0,0,<class 'sklearn.decomposition._pca.PCA'>
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
9,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),f1_score,nested cv,69.230769,9.166443,71.391941,6.264593,73.333333,17.492367,71.258741,9.420480,90.0,0,<class 'sklearn.decomposition._pca.PCA'>


In [38]:
for n_comps in np.arange(100, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, TruncatedSVD, n_outer_folds=13, n_inner_folds=6, n_components=n_comps, mod_selection_score=f1_score))

scores_df

,algorythm,best n_neighbors,best contamination,best metric,n_components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class,decomposition
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.0,0,<class 'sklearn.decomposition._pca.PCA'>
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.0,0,<class 'sklearn.decomposition._pca.PCA'>
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.0,0,<class 'sklearn.decomposition._pca.PCA'>
9,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),f1_score,nested cv,69.230769,9.166443,71.391941,6.264593,73.333333,17.492367,71.258741,9.420480,90.0,0,<class 'sklearn.decomposition._pca.PCA'>


In [7]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

file_handler = logging.FileHandler('../logs/lof_fastica_nested_cv.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))

logger = logging.getLogger()

logger.handlers = []
logger.addHandler(file_handler)

In [23]:
for n_comps in np.arange(90, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, FastICA, n_outer_folds=10, n_inner_folds=6, n_components=n_comps))

scores_df

,algorythm,best n_neighbors,best contamination,best metric,n_components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class,decomposition
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.000000,0,<class 'sklearn.decomposition._pca.PCA'>
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.000000,0,<class 'sklearn.decomposition._pca.PCA'>
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
9,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),f1_score,nested cv,69.230769,9.166443,71.391941,6.264593,73.333333,17.492367,71.258741,9.420480,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>


In [25]:
for n_comps in np.arange(90, 59, -5):
    scores_df = add_record(scores_df, nested_cv_extended(X, random_seed, FastICA, n_outer_folds=10, n_inner_folds=6, n_components=n_comps, mod_selection_score=f1_score))

scores_df

,algorythm,best n_neighbors,best contamination,best metric,n_components,best scaler,score used for model selection,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,f1 mean,f1 std,best overall accuracy,class,decomposition
0,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,69.423077,8.656457,70.769231,20.471111,68.714363,12.127024,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
1,LocalOutlierFactor,40,0.488864,manhattan,95,MinMaxScaler(),accuracy_score,nested cv,70.000000,13.587324,72.628205,11.387907,73.846154,20.121263,71.977766,13.683727,100.000000,0,<class 'sklearn.decomposition._pca.PCA'>
2,LocalOutlierFactor,35,0.177045,cosine,90,StandardScaler(),accuracy_score,nested cv,69.230769,12.064913,71.346154,8.900797,73.846154,20.121263,71.278465,12.406478,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
3,LocalOutlierFactor,35,0.199318,cosine,85,StandardScaler(),accuracy_score,nested cv,67.692308,10.490909,69.075092,8.972378,74.615385,16.305757,70.883988,9.712261,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
4,LocalOutlierFactor,35,0.199318,cosine,80,StandardScaler(),accuracy_score,nested cv,68.461538,11.665962,70.576923,12.400823,73.333333,18.673991,70.719025,12.162156,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
5,LocalOutlierFactor,40,0.366364,manhattan,75,RobustScaler(),accuracy_score,nested cv,68.461538,12.307692,71.144689,13.115331,73.076923,16.664694,70.898332,11.200656,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
6,LocalOutlierFactor,35,0.199318,cosine,70,StandardScaler(),accuracy_score,nested cv,67.692308,9.730085,69.423077,7.962147,73.333333,17.492367,70.299444,9.873557,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
7,LocalOutlierFactor,40,0.366364,cosine,65,StandardScaler(),accuracy_score,nested cv,67.692308,13.674145,70.100733,13.995670,73.076923,15.930487,70.714841,12.889715,100.000000,0,<class 'sklearn.decomposition._pca.PCA'>
8,LocalOutlierFactor,35,0.210455,cosine,60,StandardScaler(),accuracy_score,nested cv,66.923077,11.357556,68.946886,11.252444,73.076923,15.930487,69.968920,10.821103,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>
9,LocalOutlierFactor,35,0.199318,cosine,100,StandardScaler(),f1_score,nested cv,69.230769,9.166443,71.391941,6.264593,73.333333,17.492367,71.258741,9.420480,90.000000,0,<class 'sklearn.decomposition._pca.PCA'>


In [26]:
file_path = '../pickles/lof_df.pickle'

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)